### Bibliotecas

In [ ]:
#!pip install google-auth
#!pip install --upgrade google-api-python-client
#!pip install --upgrade google-cloud-iam
#!pip install --upgrade oauth2client
#!pip install google-cloud-storage
#!pip install --upgrade google-cloud-bigquery
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#!pip install --upgrade google-auth
#!pip install pandas-gbq 


In [33]:
#Bibliotecas adicionais
import itertools as it
import pandas as pd 
import numpy as np
from tqdm import tqdm

#GoogleAnalytics
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

#BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account

#GoogleSheets
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError

### Google Sheets

In [20]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

SAMPLE_SPREADSHEET_ID = '1e9lvuMR_aog6drLWJn6VdoZoVWtl7K-vCV54fhoHfRM'
SAMPLE_RANGE_NAME = 'Seller!B2:B'
RANGE_NAME = 'Seller!C2:C'


def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
       
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)
        
        global sheet
        sheet = service.spreadsheets()
        global resultSheets
        resultSheets = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        global resultSheelts2
        resultSheelts2 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        
    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()

sellers_names = pd.DataFrame(resultSheets)
sellers_names = sellers_names['values']
sellers_names = list(sellers_names)
sellers_names

sellers_id = pd.DataFrame(resultSheelts2)
sellers_id = sellers_id['values']
sellers_id = list(sellers_id)
sellers_ids = []
for se in sellers_id:
    sellers_ids.extend(se)

### Google Analytics

In [34]:
#Credenciais
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'MY_KEY.json'

#Informações dos Sellers
sellers_ids = sellers_ids
sellers_names = sellers_names
VIEW_ID = '0'

#Variáveis que unificam as requisições de cada Seller.
lista = []
resultado_unido = []
rows_len = []

for seller in tqdm(sellers_ids):
    VIEW_ID = seller
    
    #Corpo da requisição para API.
    body =   {
            'reportRequests': [
            {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': 'yesterday', 'endDate': 'today'}],
              'metrics': [{'expression': 'ga:sessions'},
                         {'expression':'ga:users'}],
              'dimensions': [{'name':'ga:date'},
                            {'name': 'ga:eventCategory'},
                            {'name':'ga:eventLabel'},
                            {'name':'ga:eventAction'},
                            ]
            }]
          }

    #Requisição
    def inicializacao_retorno_analytics():
      credentials = ServiceAccountCredentials.from_json_keyfile_name(
          KEY_FILE_LOCATION, SCOPES)
      analytics = build('analyticsreporting', 'v4', credentials=credentials)
      return analytics

    def get_report(analytics, pageTokenVar):
      return analytics.reports().batchGet(body=body).execute()
    

    # Formatando o resultado da requisição

    def handle_report(analytics,pagetoken,rows):  
        response = get_report(analytics, pagetoken)

        #Header, Dimentions Headers, Metric Headers 
        columnHeader = response.get("reports")[0].get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        #Pagination
        pagetoken = response.get("reports")[0].get('nextPageToken', None)

        #Rows
        rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
        rows = rows + rowsNew
        #print("len(rows): " + str(len(rows)))

       
        if pagetoken != None:
            return handle_report(analytics,pagetoken,rows)
        else:
          
            nicerows=[]
            for row in rows:
                dic={}
                dimensions = row.get('dimensions', [])
                dateRangeValues = row.get('metrics', [])

                for header, dimension in zip(dimensionHeaders, dimensions):
                    dic[header] = dimension

                for i, values in enumerate(dateRangeValues):
                    for metric, value in zip(metricHeaders, values.get('values')):
                        if ',' in value or ',' in value:
                            dic[metric.get('name')] = float(value)
                        else:
                            dic[metric.get('name')] = float(value)
                nicerows.append(dic)

            return nicerows

  
    def main():

        analytics = inicializacao_retorno_analytics()

        global dataframe
        dataframe = []

        global rows
        rows = []
        rows = handle_report(analytics,'0',rows)


    if __name__ == '__main__':
      main()
    
    # Adicionando o resultado já formatado da requisição de cada Seller em uma única array
    lista.extend(rows)  
    
    # Criando a array com o nome dos Sellers de acordo com o tamanho da array do retorno de cada requição.
    rows_len.append(len(rows))
for a,b in zip(sellers_names,rows_len):
        resultado_unido.extend(list(it.repeat(a,b)))
        
 #Montando o DataFrame
tabela_sem_nome_seller = pd.DataFrame(lista)
tabela_nomes_sellers = pd.DataFrame(resultado_unido)
tabela_pronta = tabela_sem_nome_seller.join(tabela_nomes_sellers)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:24<00:00,  1.22s/it]


In [35]:
#Formatando o DataFrame
tabela_pronta.columns = ['Data','Categoria_do_Evento','Acao_do_evento','Rotulo_do_evento', 'Sessoes', 'Usuarios', 'Sellers']
tabela_pronta = tabela_pronta.astype({
    'Categoria_do_Evento':'string',
    'Acao_do_evento':'string',
    'Rotulo_do_evento':'string',
    'Data':'datetime64',
    'Sessoes':'int',
    'Usuarios':'int',
    'Sellers':'string'
})

tabela_pronta



,Data,Categoria_do_Evento,Acao_do_evento,Rotulo_do_evento,Sessoes,Usuarios,Sellers
0,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO
DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO,DADO OCULTO
2,2022-11-17,Ecommerce Helper,(not set),productClick,2,2,LOJA.BARIGUISEMINOVOS.COM.BR
3,2022-11-17,Ecommerce Helper,(not set),productDetails,5,5,LOJA.BARIGUISEMINOVOS.COM.BR
4,2022-11-17,Ecommerce Helper,(not set),productImpressions,5,5,LOJA.BARIGUISEMINOVOS.COM.BR
...,...,...,...,...,...,...,...
1690,2022-11-17,Ecommerce Helper,(not set),productImpressions,1,1,PECAS.AVERSA.COM.BR
1691,2022-11-17,Ecommerce Helper,(not set),promotionClick,1,1,PECAS.AVERSA.COM.BR
1692,2022-11-17,Ecommerce Helper,(not set),promotionView,3,3,PECAS.AVERSA.COM.BR
1693,2022-11-18,Ecommerce Helper,(not set),promotionClick,1,1,PECAS.AVERSA.COM.BR


### Big Query

In [28]:
#Enviado os dados para o BigQuery
key_path = 'GBQ.json'
credentials = service_account.Credentials.from_service_account_file(
key_path, scopes = ['https://www.googleapis.com/auth/cloud-platform'])
tabela_pronta.to_gbq(credentials = credentials,
                    destination_table = 'Retail.categories',
                    if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
